In [6]:
%pip install pandas
%pip install ultralytics
%pip install tqdm


import os
import pandas as pd
from ultralytics import YOLO  
from dotenv import load_dotenv
from tqdm import tqdm  
from pathlib import Path


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached ultralytics-8.4.6-py3-none-any.whl.metadata (38 kB)
  Using cached matplotlib-3.10.8-cp314-cp314-win_amd64.whl.metadata (52 kB)
  Using cached opencv_python-4.13.0.90-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached pillow-12.1.0-cp314-cp314-win_amd64.whl.metadata (9.0 kB)
  Using cached pyyaml-6.0.3-cp314-cp314-win_amd64.whl.metadata (2.4 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached torch-2.9.1-cp314-cp314-win_amd64.whl.metadata (30 kB)
  Using cached torchvision-0.24.1-cp314-cp314-win_amd64.whl.metadata (5.9 kB)
  Using cached polars-1.37.1-py3-none-any.whl.metadata (10 kB)
  Using cached ultralytics_thop-2.0.18-py3-none-any.whl.metadata (14 kB)
  Using cached contourpy-1.3.3-cp314-cp314-win_amd64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.61.1-cp314-cp314-win_amd64.whl.metadata (116 kB)
  Using cached kiwisolver-1.4.9-cp314-cp314-win_amd64.whl.metadata 


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Note: you may need to restart the kernel to use updated packages.
Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\user\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
load_dotenv()

IMAGES_PATH = os.getenv("IMAGES_PATH", "../data/raw/images ")
OUTPUT_CSV = os.getenv("YOLO_OUTPUT", "../data/processed/yolo_detections.csv")
CONF_THRESHOLD = float(os.getenv("YOLO_CONF_THRESHOLD", 0.25))
MODEL_NAME = os.getenv("YOLO_MODEL", "yolov8n.pt")


In [14]:
try:
    model = YOLO(MODEL_NAME)
except Exception as e:
    raise RuntimeError(f"Failed to load YOLO model: {e}")



In [15]:
def classify_image(objects):
    has_person = "person" in objects
    has_product = any(obj in objects for obj in ["bottle", "cup", "container"])

    if has_person and has_product:
        return "promotional"
    if has_product and not has_person:
        return "product_display"
    if has_person and not has_product:
        return "lifestyle"
    return "other"


In [19]:
records = []

image_files = list(Path(IMAGES_PATH).rglob("*.jpg"))

for img_path in tqdm(image_files, desc="Running YOLO Detection"):
    try:
        results = model(img_path, conf=CONF_THRESHOLD, verbose=False)
        boxes = results[0].boxes

        detected = [model.names[int(cls)] for cls in boxes.cls] if boxes else []
        confidence = float(boxes.conf.mean()) if boxes else 0.0

        category = classify_image(detected)

        records.append({
            "message_id": img_path.stem,
            "image_path": str(img_path),
            "detected_objects": detected,
            "confidence_score": confidence,
            "image_category": category
        })

    except Exception as e:
        print(f"Skipping {img_path}: {e}")




Running YOLO Detection: 0it [00:00, ?it/s]


In [20]:
df = pd.DataFrame(records)
df.to_csv(OUTPUT_CSV, index=False)
df.head()


""
